# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import keras

Using TensorFlow backend.


In [0]:
(trainX,trainY),(testX,testY)=keras.datasets.cifar10.load_data()

In [3]:
trainX.shape

(50000, 32, 32, 3)

In [0]:
trainX = trainX.reshape(trainX.shape[0], 32, 32, 3).astype('float32')
testX = testX.reshape(testX.shape[0], 32, 32, 3).astype('float32')
trainX /= 255
testX /=255

In [5]:
trainY.shape

(50000, 1)

In [0]:
import numpy as np

In [0]:
trainY=np.ndarray.flatten(trainY)
testY=np.ndarray.flatten(testY)

In [0]:
x_train_lt5 = trainX[trainY < 5]
y_train_lt5 = trainY[trainY < 5]
x_test_lt5 = testX[testY < 5]
y_test_lt5 = testY[testY < 5]

x_train_gt5 = trainX[trainY >= 5]
y_train_gt5 = trainY[trainY >= 5] - 5  # make classes start at 0 for
x_test_gt5 = testX[testY >= 5]         # np_utils.to_categorical
y_test_gt5 = testY[testY >= 5] - 5

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_lt5=keras.utils.to_categorical(y_train_lt5,5)
y_test_lt5=keras.utils.to_categorical(y_test_lt5,5)
y_train_gt5=keras.utils.to_categorical(y_train_gt5,5)
y_test_gt5=keras.utils.to_categorical(y_test_gt5,5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

In [23]:
#Define model
model=Sequential()

#1st conv layer
model.add(keras.layers.Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(keras.layers.Activation('relu'))

#2nd conv layer
model.add(keras.layers.Convolution2D(32, 3, 3))
model.add(keras.layers.Activation('relu'))

#Max Pooling

model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

#Drop Out
model.add(keras.layers.Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(keras.layers.Activation('relu'))
# Batch normalization layer added here
model.add(BatchNormalization())
#Drop Out
model.add(keras.layers.Dropout(0.25))
model.add(Dense(5))
model.add(keras.layers.Activation('softmax'))
# Loss and Optimizer
adam = Adam(lr=0.0006, beta_1=0.9, beta_2=0.999, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(x_train_lt5, y_train_lt5, batch_size=32, nb_epoch=10, 
              validation_data=(x_test_lt5, y_test_lt5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3...)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 282us/step - loss: 0.9797 - acc: 0.6120 - val_loss: 0.8127 - val_acc: 0.6820
Epoch 2/10
25000/25000 [==============================] - 5s 219us/step - loss: 0.7578 - acc: 0.7100 - val_loss: 0.7550 - val_acc: 0.7092
Epoch 3/10
25000/25000 [==============================] - 5s 219us/step - loss: 0.6804 - acc: 0.7409 - val_loss: 0.6815 - val_acc: 0.7436
Epoch 4/10
25000/25000 [==============================] - 5s 219us/step - loss: 0.6187 - acc: 0.7677 - val_loss: 0.6265 - val_acc: 0.7650
Epoch 5/10
25000/25000 [==============================] - 5s 217us/step - loss: 0.5638 - acc: 0.7881 - val_loss: 0.6030 - val_acc: 0.7686
Epoch 6/10
25000/25000 [==============================] - 5s 219us/step - loss: 0.5179 - acc: 0.8049 - val_loss: 0.6212 - val_acc: 0.7728
Epoch 7/10
25000/25000 [==============================] - 5s 219us/step - loss: 0.4819 - acc: 0.8192 - val_loss: 0.6649 - v

In [0]:
#Data Agumentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True)   # flip images horizontally

validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(x_train_lt5[:40000],y_train_lt5[:40000], batch_size=32)
validation_generator = validation_datagen.flow(x_train_lt5[40000:], y_train_lt5[40000:], batch_size=32)

In [27]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_generator,    
                    validation_data=validation_generator,
                    validation_steps=len(trainX[40000:]) / 32,
                    steps_per_epoch=len(trainY[:40000]) / 32,
                    epochs=15,
                    verbose=2)

Epoch 1/15
 - 19s - loss: 0.5750 - acc: 0.7845
Epoch 2/15
 - 19s - loss: 0.5676 - acc: 0.7901
Epoch 3/15
 - 19s - loss: 0.5584 - acc: 0.7919
Epoch 4/15
 - 19s - loss: 0.5407 - acc: 0.8009
Epoch 5/15
 - 19s - loss: 0.5530 - acc: 0.7933
Epoch 6/15
 - 19s - loss: 0.5429 - acc: 0.7963
Epoch 7/15
 - 19s - loss: 0.5317 - acc: 0.8038
Epoch 8/15
 - 19s - loss: 0.5317 - acc: 0.8024
Epoch 9/15
 - 19s - loss: 0.5215 - acc: 0.8063
Epoch 10/15
 - 19s - loss: 0.5184 - acc: 0.8086
Epoch 11/15
 - 19s - loss: 0.5125 - acc: 0.8105
Epoch 12/15
 - 19s - loss: 0.5130 - acc: 0.8097
Epoch 13/15
 - 19s - loss: 0.5048 - acc: 0.8147
Epoch 14/15
 - 19s - loss: 0.4984 - acc: 0.8155
Epoch 15/15
 - 19s - loss: 0.5009 - acc: 0.8155


In [28]:
score = model.evaluate(x_test_lt5, y_test_lt5, batch_size=128, verbose=0)
print(model.metrics_names)
print(score)

['loss', 'acc']
[0.4826273998260498, 0.8226]


In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
activation_35 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_36 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 6272)              0         
__________

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [31]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_19
conv2d_19is not trainable

activation_35
activation_35is not trainable

conv2d_20
conv2d_20is not trainable

activation_36
activation_36is not trainable

max_pooling2d_10
max_pooling2d_10is not trainable

dropout_17
dropout_17is not trainable

flatten_9
flatten_9is not trainable

dense_17
dense_17 is trainable

activation_37
activation_37is not trainable

batch_normalization_4
batch_normalization_4is not trainable

dropout_18
dropout_18is not trainable

dense_18
dense_18 is trainable

activation_38
activation_38is not trainable



### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [33]:
model.fit(x_train_gt5, y_train_gt5, batch_size=32, nb_epoch=10, 
              validation_data=(x_test_gt5, y_test_gt5))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
  704/25000 [..............................] - ETA: 6s - loss: 0.3858 - acc: 0.8594

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 6s 222us/step - loss: 0.3871 - acc: 0.8601 - val_loss: 0.4084 - val_acc: 0.8508
Epoch 2/10
25000/25000 [==============================] - 5s 219us/step - loss: 0.3564 - acc: 0.8726 - val_loss: 0.4470 - val_acc: 0.8370
Epoch 3/10
25000/25000 [==============================] - 5s 218us/step - loss: 0.3323 - acc: 0.8792 - val_loss: 0.4144 - val_acc: 0.8480
Epoch 4/10
25000/25000 [==============================] - 5s 219us/step - loss: 0.3068 - acc: 0.8902 - val_loss: 0.3806 - val_acc: 0.8618
Epoch 5/10
25000/25000 [==============================] - 5s 218us/step - loss: 0.2750 - acc: 0.9011 - val_loss: 0.4174 - val_acc: 0.8492
Epoch 6/10
25000/25000 [==============================] - 5s 218us/step - loss: 0.2575 - acc: 0.9051 - val_loss: 0.3936 - val_acc: 0.8592
Epoch 7/10
25000/25000 [==============================] - 5s 218us/step - loss: 0.2402 - acc: 0.9120 - val_loss: 0.4911 - val_acc: 0.8330
Epoch 8/10
25000/25000 [=====================

## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [0]:
import pandas as pd
data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/tweets.csv', encoding = "ISO-8859-1").dropna()

In [37]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [68]:
data.shape

(3291, 3)

In [69]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [0]:
data = data[(data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [71]:
data.shape

(3191, 3)

### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
from sklearn.model_selection import train_test_split
X = data.tweet_text
y = data['is_there_an_emotion_directed_at_a_brand_or_product']

In [0]:
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

### 8. Find number of different words in vocabulary

In [87]:
X_dtm.shape

(3191, 5648)

#### Tip: To see all available functions for an Object use dir

In [58]:
dir(vect)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_sort_features',
 '_stop_words_id',
 '_validate_custom_analyzer',
 '_validate_params',
 '_validate_vocabulary',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',
 'build_preprocessor',
 'build_tokenizer',
 'decode',
 'decode_error',
 'dtype',
 'encoding',
 'fit',
 'fit_transform',
 'fixed_vocabulary_',
 'get_feature_names',
 'get_params',
 'get_stop_words',
 'input',
 'inverse_transf

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [89]:
pd.value_counts(data['is_there_an_emotion_directed_at_a_brand_or_product'])

Positive emotion    2672
Negative emotion     519
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [0]:
data['label'] = data.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [0]:
X = data.tweet_text
y=data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [0]:
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [93]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
nb=MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print (accuracy_score(y_test, y_pred_class))

0.8471177944862155


In [94]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print (accuracy_score(y_test, y_pred_class))

0.8471177944862155


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [0]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(X_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ',accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [100]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  24855
Accuracy:  0.8558897243107769


### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [101]:
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  4681
Accuracy:  0.8533834586466166


### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [102]:
vect = CountVectorizer(stop_words='english',max_features =300)
tokenize_test(vect)

Features:  300
Accuracy:  0.8107769423558897


### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [103]:
vect = CountVectorizer(ngram_range=(1, 2),max_features =15000)
tokenize_test(vect)

Features:  15000
Accuracy:  0.8533834586466166


### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [104]:
vect = CountVectorizer(ngram_range=(1, 2),min_df = 2)
tokenize_test(vect)

Features:  7764
Accuracy:  0.8583959899749374
